In [1]:
import sklearn
import matplotlib.pyplot as plt
import h5py
import numpy as np
import scipy
import seaborn as sns
import math
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, RepeatVector, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import KFold

2023-11-07 22:11:03.368353: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


ModuleNotFoundError: No module named 'tensorflow.keras.wrappers'

In [2]:
print(tf.config.experimental.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU')]


2023-11-06 21:46:33.253128: I tensorflow/stream_executor/platform/default/dso_loader.cc:50] Successfully opened dynamic library libcuda.so.1
2023-11-06 21:46:33.339439: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1674] Found device 0 with properties: 
name: NVIDIA A100 80GB PCIe major: 8 minor: 0 memoryClockRate(GHz): 1.41
pciBusID: 0000:01:00.0
2023-11-06 21:46:33.340596: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1674] Found device 1 with properties: 
name: NVIDIA A100 80GB PCIe major: 8 minor: 0 memoryClockRate(GHz): 1.41
pciBusID: 0000:25:00.0
2023-11-06 21:46:33.341753: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1674] Found device 2 with properties: 
name: NVIDIA A100 80GB PCIe major: 8 minor: 0 memoryClockRate(GHz): 1.41
pciBusID: 0000:c1:00.0
2023-11-06 21:46:33.342906: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1674] Found device 3 with properties: 
name: NVIDIA A100 80GB PCIe major: 8 minor: 0 memoryClockRate(GHz): 1.41
pciBusID: 0000:e1:00.0
202

In [3]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  4


In [2]:
#Prepare Training Data:
path_to_X = "./Normalised/X.npy"
path_to_Y = "./Normalised/Y.npy"

X = np.load(path_to_X)
Y = np.load(path_to_Y)

print(len(X), len(Y))

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.20, random_state=42)
# Create a validation set from the X_train and Y_train:
#X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.17647, random_state = 42)

print("X_Train length: ",len(X_train))
#print("X_Val length: ",len(X_val))
print("X_Test length: ",len(X_test))
print("Original X_train Shape: ",X_train.shape)
print("Original Y_train Shape: ",Y_train.shape)
# print("Original X_val Shape: ",X_val.shape)
# print("Original Y_val Shape: ",Y_val.shape)

# Reshape the input and output data for the encoder and decoder
X_train_reshaped = X_train.reshape(X_train.shape[0], X_train.shape[1], 1) # (No of Data, 50) to (No of data, 50,1)
print("New X-train shape: ", X_train_reshaped.shape)
Y_train_reshaped = Y_train.reshape(Y_train.shape[0], Y_train.shape[1], 1)
print("New Y-train shape: ", Y_train_reshaped.shape)
# Define the input shape for the encoder
input_shape = X_train_reshaped.shape[1:]  # Shape: (sequence_length, 1) # 50,1
print("Input_shape: ", input_shape)
X_test_reshaped = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
Y_test_reshaped = Y_test.reshape(Y_test.shape[0], Y_test.shape[1], 1)
# X_val_reshaped = X_val.reshape(X_val.shape[0], X_val.shape[1], 1)
# Y_val_reshaped = Y_val.reshape(Y_val.shape[0], Y_val.shape[1], 1)
# print(f"X_test_reshaped:{X_test_reshaped.shape} ; Y_test_reshaped:{Y_test_reshaped.shape} ; X_val_reshaped: {X_val_reshaped.shape} ; Y_val_reshaped: {Y_val_reshaped.shape} ")

62163 62163
X_Train length:  49730
X_Test length:  12433
Original X_train Shape:  (49730, 50)
Original Y_train Shape:  (49730, 81)
New X-train shape:  (49730, 50, 1)
New Y-train shape:  (49730, 81, 1)
Input_shape:  (50, 1)


In [3]:
# Define the function to create the model
def create_autoencoder_model(units=32, optimizer='adam'):
    # Define the encoder
    encoder_inputs = Input(shape=input_shape)
    encoder_lstm, state_h, state_c = LSTM(units, return_state=True)(encoder_inputs)  # Set return_state to True
    encoder_states = [state_h, state_c]  # Capture the final states
    encoder_model = Model(encoder_inputs, encoder_lstm)  # Create the encoder model

    # Define the input shape for the decoder (with 40 units to match encoder LSTM units)
    decoder_inputs = Input(shape=(units,))
    decoder_lstm = LSTM(units, return_sequences=True)(RepeatVector(Y_train_reshaped.shape[1])(decoder_inputs))
    decoder_outputs = Dense(1, activation='sigmoid')(decoder_lstm)
    decoder_model = Model(decoder_inputs, decoder_outputs)

    # Create the full autoencoder model
    autoencoder_outputs = decoder_model(encoder_lstm)
    autoencoder = Model(encoder_inputs, autoencoder_outputs)
    autoencoder.compile(optimizer=optimizer, loss='mean_squared_error',metrics=['mae', 'mape'])
    #autoencoder.summary()
    return autoencoder

In [7]:
#strategy = tf.distribute.MirroredStrategy(devices=["/GPU:0", "/GPU:1","/GPU:2", "/GPU:3"])

In [5]:
#validation_data = (X_val_reshaped, Y_val_reshaped)
#with strategy.scope():
model = KerasRegressor(build_fn=create_autoencoder_model, epochs=200, batch_size=32, verbose=1)
# Define the hyperparameter grid
param_grid = {'units': [32, 64, 128], 'optimizer': ['adam', 'rmsprop']}
# Define callbacks for early stopping and model checkpointing
callbacks = [EarlyStopping(monitor='val_loss', patience=30, verbose = 1), ModelCheckpoint(filepath='./Saved_Models_Data/ED1/MC_{epoch:02d}_{val_loss:.2f}_{units}_{optimizer}', monitor='val_loss', mode = 'min', save_best_only=True, verbose = 1)]

# Perform grid search with callbacks
grid = GridSearchCV(estimator=model, param_grid=param_grid, return_train_score=True, scoring='neg_mean_squared_error', cv=3)
grid_result = grid.fit(X_train_reshaped, Y_train_reshaped, validation_split = 0.2 , callbacks=callbacks)

# Print the best parameters and the best score
print("Best Parameters: ", grid_result.best_params_)
print("Best Score: ", grid_result.best_score_)
with open('./Saved_Models_Data/ED1/best_parameters.txt', 'w') as file:
    file.write("Best Parameters: " + str(grid_result.best_params_) + "\n")
    file.write("Best Score: " + str(grid_result.best_score_) + "\n")
# Confirm that the best parameters have been saved
print("Best parameters saved to file successfully.")

# Display the training history
print("Training History:")
print(grid_result.history)
# Save the displayed history to a text file
with open('./Saved_Models_Data/ED1/training_history.txt', 'w') as hfile:
    hfile.write("Training History:\n")
    for key, value in grid_result.history.items():
        file.write("%s : %s\n" % (key, value))
# Confirm that the training history has been saved
print("Training history saved to file successfully.")

# Evaluate the best model on the test data
best_model = grid_result.best_estimator_
test_score = best_model.score(X_test_reshaped, Y_test_reshaped)
print("Test Score: ", test_score)
#Save the best model's evaluated score on the test data to a text file
with open('./Saved_Models_Data/ED1/test_score.txt', 'w') as tfile:
    tfile.write("Test Score: " + str(test_score) + "\n")
# Confirm that the test score has been saved
print("Test score saved to file successfully.")

#Save best model:
overall_best_model = grid_result.best_estimator_.model
overall_best_model.save('./Saved_Models_Data/ED1/overall_best_model.h5')
print("Overall best model saved to file successfully")

# Show results for all different parameters
means = grid_result.cv_results_['mean_test_score']
params = grid_result.cv_results_['params']
for mean, param in zip(means, params):
    print("Mean score: {:.4f} with parameters: {}".format(mean, param))

# Save the results for all different parameters:
with open('parameter_results.txt', 'w') as file:
    file.write("Results for all different parameters:\n")
    for mean, param in zip(means, params):
        file.write("Mean score: {:.4f} with parameters: {}\n".format(mean, param))
print("Parameter results saved to file successfully.")


Train on 26522 samples, validate on 6631 samples
Epoch 1/200
26496/26522 [============================>.] - ETA: 0s - loss: 0.1247 - mean_absolute_error: 0.2512 - mean_absolute_percentage_error: 128369600.0000

/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.8/dist-packages/tensorflow_core/python/keras/wrappers/scikit_learn.py", line 166, in fit
    history = self.model.fit(x, y, **fit_args)
  File "/usr/local/lib/python3.8/dist-packages/tensorflow_core/python/keras/engine/training.py", line 708, in fit
    return func.fit(
  File "/usr/local/lib/python3.8/dist-packages/tensorflow_core/python/keras/engine/training_arrays.py", line 658, in fit
    return fit_loop(
  File "/usr/local/lib/python3.8/dist-packages/tensorflow_core/python/keras/engine/training_arrays.py", line 450, in model_iteration
    callb

Train on 26522 samples, validate on 6631 samples
Epoch 1/200
  992/26522 [>.............................] - ETA: 2:23 - loss: 0.1726 - mean_absolute_error: 0.3632 - mean_absolute_percentage_error: 260325440.0000

KeyboardInterrupt: 